### Importación de datos

In [3]:
import pandas as pd
import numpy as np
import pyomo.environ as pyo
import logging
from pyomo.opt import SolverStatus, TerminationCondition
import matplotlib.pyplot as plt


# Creación de Instancia Ficticia general


"""--------------------------------------Importación de datos---------------------------------------"""
def instancia():
    
    np.random.seed(42)
    regiones_de_chile = ["Región Metropolitana de Santiago","Libertador General Bernardo O’Higgins",
        "Maule","Ñuble","Biobío","La Araucanía","Los Ríos","Los Lagos"]

    # Generar datos aleatorios para 30 Proveedores
    Info_prov = pd.DataFrame({
        'Region': np.random.choice(regiones_de_chile, 30)
    }, index=['Prov{}'.format(i) for i in range(1, 31)])

    # Generar datos aleatorios para 15 Clientes
    Info_cliente = pd.DataFrame({
        'Region': np.random.choice(regiones_de_chile, 15)
    }, index=['Cliente{}'.format(i) for i in range(1, 16)])

    # Generar datos aleatorios para 10 Localizaciones
    Info_Loc = pd.DataFrame({
        'Region': np.random.choice(regiones_de_chile, 10)
    }, index=[chr(i) for i in range(65, 75)])  # ASCII de A a J

    # Generar datos aleatorios para Demanda de PLA de los clientes
    Demanda_cliente = pd.DataFrame({
        'Demanda (ton)': np.random.randint(5000, 15000, size=15)
    }, index=['Cliente{}'.format(i) for i in range(1, 16)])

    # Generar datos aleatorios para Demanda de Compost de los proveedores
    Demanda_prov = pd.DataFrame({
        'Demanda (ton)': np.random.randint(10000, 50000, size=30)
    }, index=['Prov{}'.format(i) for i in range(1, 31)])

    # Generar datos aleatorios para Capacidad de recepción de Materia Prima por localización
    Capacidad_MP = pd.DataFrame({
        'Capacidad MP (ton)': np.random.randint(50000, 200000, size=10)
    }, index=[chr(i) for i in range(65, 75)])

    # Generar datos aleatorios para Capacidad de recepción de retornos por localización
    Capacidad_r = pd.DataFrame({
        'Capacidad retornos (ton)': np.random.randint(25000, 75000, size=10)
    }, index=[chr(i) for i in range(65, 75)])

    # Generar datos aleatorios para Disponibilidad de Materia Prima en cada proveedor
    Disponibilidad_MP = pd.DataFrame({
        'Maiz': np.random.randint(0, 150000, size=30),
        'Papa': np.random.randint(0, 50000, size=30)
    }, index=['Prov{}'.format(i) for i in range(1, 31)])

    # Generar datos aleatorios para Matriz de distancia entre proveedor y localización
    Distancias_prov = pd.DataFrame(np.random.randint(50, 1000, (30, 10)), 
                                   index=['Prov{}'.format(i) for i in range(1, 31)], 
                                   columns=[chr(i) for i in range(65, 75)])

    # Generar datos aleatorios para Matriz de distancia entre localización y cliente
    Distancias_cliente = pd.DataFrame(np.random.randint(20, 700, (10, 15)), 
                                       index=[chr(i) for i in range(65, 75)], 
                                       columns=['Cliente{}'.format(i) for i in range(1, 16)])

    # Generar datos aleatorios para Inversión por localización
    Inversion = pd.DataFrame({
        'Inversión Poli': np.random.randint(500000000, 550000000, size=10),
        'Inversión Comp': np.random.randint(4500000, 5500000, size=10)
    }, index=[chr(i) for i in range(65, 75)])

    # Generar datos aleatorios para Costos de Adquisición de Materia Prima
    costos_maiz = np.random.randint(150000, 200000, size=30)
    costos_papa = np.random.randint(350000, 450000, size=30)

    Costos_adquisicion = pd.DataFrame({
        'Maiz': costos_maiz,
        'Papa': costos_papa
    }, index=['Prov{}'.format(i) for i in range(1, 31)])

    # Transponer el DataFrame para tener Maiz y Papa como índices
    Costos_adquisicion = Costos_adquisicion.T

    Info_MP = pd.read_excel('PLA data.xlsx',sheet_name='Info_MP',index_col=0)
    Info_r = pd.read_excel('PLA data.xlsx',sheet_name='Info_r',index_col=0)    
    Precios_venta = pd.read_excel('PLA data.xlsx',sheet_name='Precios_venta',index_col=0)
    Cap_prod = 20000
    Costo_transp = 21.9912
    Emision_transp = 0.00005177142857 
    cuota_PLA = 0.2
    cuota_Comp = 0.02290
    
    #Conjuntos

    m_primas = Info_MP.index.to_list()
    proveedores = Info_prov.index.to_list()
    clientes = Info_cliente.index.to_list()
    localizaciones = Info_Loc.index.to_list()
    catalogo = Precios_venta.index.to_list()

    return (Info_prov,Info_cliente,Info_Loc,Info_MP,Info_r,Demanda_cliente,Demanda_prov,Capacidad_MP,Capacidad_r,
            Disponibilidad_MP,Distancias_prov,Distancias_cliente,Costos_adquisicion,Inversion,
            Precios_venta,Cap_prod,Costo_transp,Emision_transp,cuota_PLA,cuota_Comp,m_primas,proveedores,clientes,
            localizaciones,catalogo)

##### """---------------------------------------Tabla de parametros---------------------------------------"""

try:
    (Info_prov,Info_cliente,Info_Loc,Info_MP,Info_r,Demanda_cliente,Demanda_prov,Capacidad_MP,Capacidad_r,Disponibilidad_MP,
     Distancias_prov,Distancias_cliente,Costos_adquisicion,Inversion,Precios_venta,Cap_prod,Costo_transp,
     Emision_transp,cuota_PLA,cuota_Comp,m_primas,proveedores,clientes,localizaciones,catalogo) = instancia()
    print("Data loaded")
    
except Exception as e:
    print("Ocurrio un error al momento de cargar la base de datos")
    print(e)

# Generación de ponderaciones en base a distancias
omega = {}
distancias_list = []
for l in range(len(localizaciones)):
    for p in range(len(proveedores)):
        distancias_list.append(Distancias_prov.loc[proveedores[p],localizaciones[l]])
        numerador = Distancias_prov.loc[proveedores[p],localizaciones[l]]-max(distancias_list)
        denominador = min(distancias_list) - max(distancias_list)
        if denominador == 0:
            omega[p,l] = 0
        else:
            if numerador/denominador < 0:
                omega[p,l] = 0
            else:
                omega[p,l] = numerador/denominador

omega_array = np.zeros((len(proveedores),len(localizaciones)))

for (x, y), v in omega.items():
    omega_array[x-1,y-1] = v

omega_df = pd.DataFrame(omega_array, index = proveedores, columns = localizaciones)
omega_df.to_excel("priority.xlsx")



Data loaded


### Definición del modelo

In [4]:
"""+++++++++++++++++++++++++++++++++++++++++++++++Desarrollo del programa+++++++++++++++++++++++++++++++++++++++++++++++"""

def model_1(fo, imprimir, w, FO_list):
    # Formulación del modelo
    model = pyo.ConcreteModel(name = "Supply chain PLA")

    """===============================================Variables de decisión============================================="""
    model.v = pyo.Var(localizaciones, within = pyo.Binary)
    model.w = pyo.Var(localizaciones, within = pyo.Binary)
    model.x = pyo.Var(localizaciones,clientes, within = pyo.NonNegativeReals)
    model.r = pyo.Var(localizaciones,clientes, within = pyo.NonNegativeReals)
    model.y = pyo.Var(m_primas,proveedores,localizaciones, within = pyo.NonNegativeReals)
    model.z = pyo.Var(proveedores,localizaciones, within = pyo.NonNegativeReals)
    
    """===============================================Función Objetivo=================================================="""
    if fo == 1:
        #Maximizar ganancias
        
        # Ganancias por ventas
        ingresos_cliente = sum(Precios_venta.loc["PLA_pellet","Precio"] * model.x[l,c] for l in localizaciones for c in clientes)# + sum(Precios_venta.loc["Servicio_Ex","Precio"]*model.r[l,c] for l in localizaciones for c in clientes)
        ingresos_compost = sum(Precios_venta.loc["Compost","Precio"]* model.z[p,l] for l in localizaciones for p in proveedores)
        ingresos_totales = ingresos_cliente + ingresos_compost

        #costos operacionales
        costos_transp_MP = sum(Costo_transp * Distancias_prov.loc[p,l] * model.y[m,p,l] for m in m_primas  for p in proveedores for l in localizaciones)
        costos_transp_PLA = sum(Costo_transp * Distancias_cliente.loc[l,c] * model.x[l,c] for l in localizaciones for c in clientes)
        costos_transp_r = sum(Costo_transp * Distancias_cliente.loc[l,c] * model.r[l,c] for l in localizaciones for c in clientes)
        costos_transp_Comp = sum(Costo_transp * Distancias_prov.loc[p,l] * model.z[p,l] for p in proveedores for l in localizaciones)
        costos_transp_totales = costos_transp_MP + costos_transp_PLA + costos_transp_r + costos_transp_Comp
        
        #Costos de producción
        prod_PLA = sum(Info_MP.loc[m,"Costo_Prod"]* model.y[m,p,l] for m in m_primas  for p in proveedores for l in localizaciones)
        prod_Comp = sum(Info_r.loc["PLA_r","Costo_Compostar"] * model.r[l,c] for l in localizaciones for c in clientes)
        costo_adquision_mp = sum(Costos_adquisicion.loc[m,p]* model.y[m,p,l] for m in m_primas for p in proveedores for l in localizaciones)

        costos_prod_totales = prod_PLA + prod_Comp + costo_adquision_mp
        
        
        
        #Inversión de plantas
        Inv = sum(Inversion.loc[l,"Inversión Poli"] * model.w[l] + Inversion.loc[l,"Inversión Comp"] * model.v[l] for l in localizaciones)

        # Beneficio total
        beneficio = ingresos_totales - (costos_transp_totales + costos_prod_totales + Inv)
        
        
        ## OTRAS FUNCIONES CALCULO
        
        #2
        emisiones_transp_MP = sum(Emision_transp * Distancias_prov.loc[p,l] * model.y[m,p,l] for m in m_primas  for p in proveedores for l in localizaciones)
        emisiones_transp_PLA = sum(Emision_transp * Distancias_cliente.loc[l,c] * model.x[l,c] for l in localizaciones for c in clientes)
        emisiones_transp_r = sum(Emision_transp * Distancias_cliente.loc[l,c] * model.r[l,c] for l in localizaciones for c in clientes)
        emisiones_transp_Comp = sum(Emision_transp * Distancias_prov.loc[p,l] * model.z[p,l] for p in proveedores for l in localizaciones)
        emisiones_transp_totales = emisiones_transp_MP + emisiones_transp_PLA + emisiones_transp_r + emisiones_transp_Comp
        
        #emisiones por producción
        E_prod_PLA = sum(Info_MP.loc[m,"E_Prod"]* model.y[m,p,l] for m in m_primas  for p in proveedores for l in localizaciones)
        E_prod_Comp = sum(Info_r.loc["PLA_r","E_Compostar"] * model.r[l,c] for l in localizaciones for c in clientes)
        E_prod_totales = E_prod_PLA + E_prod_Comp
        
        #3
        prioridad = sum(omega_df.loc[p,l]*model.y[m,p,l] for m in m_primas  for p in proveedores for l in localizaciones)

        
        

        # Formula de función objetivo
        model.FO = pyo.Objective(expr = beneficio, sense = pyo.maximize)
        
    elif fo == 2:
        #Minimizar emisiones de carbono
        
        #emisiones por transporte
        emisiones_transp_MP = sum(Emision_transp * Distancias_prov.loc[p,l] * model.y[m,p,l] for m in m_primas  for p in proveedores for l in localizaciones)
        emisiones_transp_PLA = sum(Emision_transp * Distancias_cliente.loc[l,c] * model.x[l,c] for l in localizaciones for c in clientes)
        emisiones_transp_r = sum(Emision_transp * Distancias_cliente.loc[l,c] * model.r[l,c] for l in localizaciones for c in clientes)
        emisiones_transp_Comp = sum(Emision_transp * Distancias_prov.loc[p,l] * model.z[p,l] for p in proveedores for l in localizaciones)
        emisiones_transp_totales = emisiones_transp_MP + emisiones_transp_PLA + emisiones_transp_r + emisiones_transp_Comp
        
        #emisiones por producción
        E_prod_PLA = sum(Info_MP.loc[m,"E_Prod"]* model.y[m,p,l] for m in m_primas  for p in proveedores for l in localizaciones)
        E_prod_Comp = sum(Info_r.loc["PLA_r","E_Compostar"] * model.r[l,c] for l in localizaciones for c in clientes)
        E_prod_totales = E_prod_PLA + E_prod_Comp
        
        #3
        prioridad = sum(omega_df.loc[p,l]*model.y[m,p,l] for m in m_primas  for p in proveedores for l in localizaciones)
        
        #1
        ingresos_cliente = sum(Precios_venta.loc["PLA_pellet","Precio"] * model.x[l,c] for l in localizaciones for c in clientes)# + sum(Precios_venta.loc["Servicio_Ex","Precio"]*model.r[l,c] for l in localizaciones for c in clientes)
        ingresos_compost = sum(Precios_venta.loc["Compost","Precio"]* model.z[p,l] for l in localizaciones for p in proveedores)
        ingresos_totales = ingresos_cliente + ingresos_compost

        #costos operacionales
        costos_transp_MP = sum(Costo_transp * Distancias_prov.loc[p,l] * model.y[m,p,l] for m in m_primas  for p in proveedores for l in localizaciones)
        costos_transp_PLA = sum(Costo_transp * Distancias_cliente.loc[l,c] * model.x[l,c] for l in localizaciones for c in clientes)
        costos_transp_r = sum(Costo_transp * Distancias_cliente.loc[l,c] * model.r[l,c] for l in localizaciones for c in clientes)
        costos_transp_Comp = sum(Costo_transp * Distancias_prov.loc[p,l] * model.z[p,l] for p in proveedores for l in localizaciones)
        costos_transp_totales = costos_transp_MP + costos_transp_PLA + costos_transp_r + costos_transp_Comp
        
        #Costos de producción
        prod_PLA = sum(Info_MP.loc[m,"Costo_Prod"]* model.y[m,p,l] for m in m_primas  for p in proveedores for l in localizaciones)
        prod_Comp = sum(Info_r.loc["PLA_r","Costo_Compostar"] * model.r[l,c] for l in localizaciones for c in clientes)
        costo_adquision_mp = sum(Costos_adquisicion.loc[m,p]* model.y[m,p,l] for m in m_primas for p in proveedores for l in localizaciones)

        costos_prod_totales = prod_PLA + prod_Comp + costo_adquision_mp

        
        #Inversión de plantas
        Inv = sum(Inversion.loc[l,"Inversión Poli"] * model.w[l] + Inversion.loc[l,"Inversión Comp"] * model.v[l] for l in localizaciones)

        # Beneficio total
        beneficio = ingresos_totales - (costos_transp_totales + costos_prod_totales + Inv)
        
        
        #Emisiones por actividad productiva
        E_total = emisiones_transp_totales + E_prod_totales
        
        #Formula de función objetivo
        model.FO = pyo.Objective(expr = E_total, sense = pyo.minimize)
        
    elif fo == 3:
        #Maximizar colaboración con proveedores locales
        
        #1
        ingresos_cliente = sum(Precios_venta.loc["PLA_pellet","Precio"] * model.x[l,c] for l in localizaciones for c in clientes)# + sum(Precios_venta.loc["Servicio_Ex","Precio"]*model.r[l,c] for l in localizaciones for c in clientes)
        ingresos_compost = sum(Precios_venta.loc["Compost","Precio"]* model.z[p,l] for l in localizaciones for p in proveedores)
        ingresos_totales = ingresos_cliente + ingresos_compost

        #costos operacionales
        costos_transp_MP = sum(Costo_transp * Distancias_prov.loc[p,l] * model.y[m,p,l] for m in m_primas  for p in proveedores for l in localizaciones)
        costos_transp_PLA = sum(Costo_transp * Distancias_cliente.loc[l,c] * model.x[l,c] for l in localizaciones for c in clientes)
        costos_transp_r = sum(Costo_transp * Distancias_cliente.loc[l,c] * model.r[l,c] for l in localizaciones for c in clientes)
        costos_transp_Comp = sum(Costo_transp * Distancias_prov.loc[p,l] * model.z[p,l] for p in proveedores for l in localizaciones)
        costos_transp_totales = costos_transp_MP + costos_transp_PLA + costos_transp_r + costos_transp_Comp
        
        #Costos de producción
        prod_PLA = sum(Info_MP.loc[m,"Costo_Prod"]* model.y[m,p,l] for m in m_primas  for p in proveedores for l in localizaciones)
        prod_Comp = sum(Info_r.loc["PLA_r","Costo_Compostar"] * model.r[l,c] for l in localizaciones for c in clientes)
        costo_adquision_mp = sum(Costos_adquisicion.loc[m,p]* model.y[m,p,l] for m in m_primas for p in proveedores for l in localizaciones)

        costos_prod_totales = prod_PLA + prod_Comp + costo_adquision_mp
        
        #2

        emisiones_transp_MP = sum(Emision_transp * Distancias_prov.loc[p,l] * model.y[m,p,l] for m in m_primas  for p in proveedores for l in localizaciones)
        emisiones_transp_PLA = sum(Emision_transp * Distancias_cliente.loc[l,c] * model.x[l,c] for l in localizaciones for c in clientes)
        emisiones_transp_r = sum(Emision_transp * Distancias_cliente.loc[l,c] * model.r[l,c] for l in localizaciones for c in clientes)
        emisiones_transp_Comp = sum(Emision_transp * Distancias_prov.loc[p,l] * model.z[p,l] for p in proveedores for l in localizaciones)
        emisiones_transp_totales = emisiones_transp_MP + emisiones_transp_PLA + emisiones_transp_r + emisiones_transp_Comp
        
        #emisiones por producción
        E_prod_PLA = sum(Info_MP.loc[m,"E_Prod"]* model.y[m,p,l] for m in m_primas  for p in proveedores for l in localizaciones)
        E_prod_Comp = sum(Info_r.loc["PLA_r","E_Compostar"] * model.r[l,c] for l in localizaciones for c in clientes)
        E_prod_totales = E_prod_PLA + E_prod_Comp
        
        
        
        
        
        #Inversión de plantas
        Inv = sum(Inversion.loc[l,"Inversión Poli"] * model.w[l] + Inversion.loc[l,"Inversión Comp"] * model.v[l] for l in localizaciones)

        # Beneficio total
        beneficio = ingresos_totales - (costos_transp_totales + costos_prod_totales + Inv)
        
        
        
        #Prioridad de transporte a colaboradores locales
        prioridad = sum(omega_df.loc[p,l]*model.y[m,p,l] for m in m_primas  for p in proveedores for l in localizaciones)
        
        #Formula de función objetivo
        model.FO = pyo.Objective(expr = prioridad, sense = pyo.maximize)

    if fo == 4:
        
        """===============================================Función multiobjetivo========================================="""
        """---------------------------FO_Ec---------------------------"""
        # Ganancias por ventas
        ingresos_cliente_MO = sum(Precios_venta.loc["PLA_pellet","Precio"] * model.x[l,c] for l in localizaciones for c in clientes)
        ingresos_compost_MO = sum(Precios_venta.loc["Compost","Precio"]* model.z[p,l] for l in localizaciones for p in proveedores)
        ingresos_totales_MO = ingresos_cliente_MO + ingresos_compost_MO
        
        
        #costos operacionales
        costos_transp_MP_MO = sum(Costo_transp * Distancias_prov.loc[p,l] * model.y[m,p,l] for m in m_primas  for p in proveedores for l in localizaciones)
        costos_transp_PLA_MO = sum(Costo_transp * Distancias_cliente.loc[l,c] * model.x[l,c] for l in localizaciones for c in clientes)
        costos_transp_r_MO = sum(Costo_transp * Distancias_cliente.loc[l,c] * model.r[l,c] for l in localizaciones for c in clientes)
        costos_transp_Comp_MO = sum(Costo_transp * Distancias_prov.loc[p,l] * model.z[p,l] for p in proveedores for l in localizaciones)
        costos_transp_totales_MO = costos_transp_MP_MO + costos_transp_PLA_MO + costos_transp_r_MO + costos_transp_Comp_MO
        
        #Costos de producción
        prod_PLA_MO = sum(Info_MP.loc[m,"Costo_Prod"]* model.y[m,p,l] for m in m_primas  for p in proveedores for l in localizaciones)
        prod_Comp_MO = sum(Info_r.loc["PLA_r","Costo_Compostar"] * model.r[l,c] for l in localizaciones for c in clientes)
        costo_adquision_mp_mo = sum(Costos_adquisicion.loc[m,p]* model.y[m,p,l] for m in m_primas for p in proveedores for l in localizaciones)

        
        costos_prod_totales_MO = prod_PLA_MO + prod_Comp_MO + costo_adquision_mp_mo
        
        #Inversión de plantas
        Inv_MO = sum(Inversion.loc[l,"Inversión Poli"] * model.w[l] + Inversion.loc[l,"Inversión Comp"] * model.v[l] for l in localizaciones)

        beneficio_MO = ingresos_totales_MO - (costos_transp_totales_MO + costos_prod_totales_MO + Inv_MO)
        
        """---------------------------FO_Am---------------------------"""
        #Minimizar emisiones de carbono
        
        #emisiones por transporte
        emisiones_transp_MP_MO = sum(Emision_transp * Distancias_prov.loc[p,l] * model.y[m,p,l] for m in m_primas  for p in proveedores for l in localizaciones)
        emisiones_transp_PLA_MO = sum(Emision_transp * Distancias_cliente.loc[l,c] * model.x[l,c] for l in localizaciones for c in clientes)
        emisiones_transp_r_MO = sum(Emision_transp * Distancias_cliente.loc[l,c] * model.r[l,c] for l in localizaciones for c in clientes)
        emisiones_transp_Comp_MO = sum(Emision_transp * Distancias_prov.loc[p,l] * model.z[p,l] for p in proveedores for l in localizaciones)
        emisiones_transp_totales_MO = emisiones_transp_MP_MO + emisiones_transp_PLA_MO + emisiones_transp_r_MO + emisiones_transp_Comp_MO
        
        #emisiones por producción
        E_prod_PLA_MO = sum(Info_MP.loc[m,"E_Prod"]* model.y[m,p,l] for m in m_primas  for p in proveedores for l in localizaciones)
        E_prod_Comp_MO = sum(Info_r.loc["PLA_r","E_Compostar"] * model.r[l,c] for l in localizaciones for c in clientes)
        E_prod_totales_MO = E_prod_PLA_MO + E_prod_Comp_MO
        
        #Emisiones por actividad productiva
        E_total_MO = emisiones_transp_totales_MO + E_prod_totales_MO
        
        """---------------------------FO_So---------------------------"""
        # Maximizar colaboración con proveedores locales
        prioridad_MO = sum(omega_df.loc[p,l]*model.y[m,p,l] for m in m_primas  for p in proveedores for l in localizaciones)
        
        """---------------------------FO_MO---------------------------"""
        
        FO_pond = -w[0]*((beneficio_MO-FO_list[0])/FO_list[0]) + w[1]*((E_total_MO-FO_list[1])/FO_list[1]) - w[2]*((prioridad_MO-FO_list[2])/FO_list[2])
        
        model.FO = pyo.Objective(expr = FO_pond, sense = pyo.minimize)

    """===============================================Restricciones====================================================="""
    #restricción de demanda máxima
    model.R1 = pyo.ConstraintList()
    for c in clientes:
        model.R1.add(expr = sum(model.x[l,c] for l in localizaciones) <= Demanda_cliente.loc[c,"Demanda (ton)"] )
    
    model.R2 = pyo.ConstraintList()
    for p in proveedores:
        model.R2.add(expr = sum(model.z[p,l] for l in localizaciones) <= Demanda_prov.loc[p,"Demanda (ton)"] )
    
    #restricción de demanda mínima
    model.R3 = pyo.Constraint(expr = cuota_PLA * sum(Demanda_cliente.loc[c,"Demanda (ton)"] for c in clientes) <=
                              sum(model.x[l,c] for l in localizaciones for c in clientes))
    
    model.R4 = pyo.Constraint(expr = cuota_Comp * sum(Demanda_prov.loc[p,"Demanda (ton)"] for p in proveedores) <=
                              sum(model.z[p,l] for l in localizaciones for p in proveedores))
    
    #Restricción de balance de flujo de producción
    model.R5 = pyo.ConstraintList()
    for l in localizaciones:
        model.R5.add(sum(Info_MP.loc[m,"Tasa_fermentación"]*Info_MP.loc[m,"Tasa_polimerizacion"]*model.y[m,p,l] for m in m_primas for p in proveedores) ==
                     sum(model.x[l,c] for c in clientes))
    
    #Restricción de balance de flujo de compostaje
    model.R6 = pyo.ConstraintList()
    for l in localizaciones:
        model.R6.add(sum (Info_r.loc["PLA_r","Tasa_Compost"] * model.r[l,c] for c in clientes) ==
                     sum(model.z[p,l] for p in proveedores))
    
    #Restricción de disponibilidad de materia prima
    model.R7 = pyo.ConstraintList()
    for p in proveedores:
        for m in m_primas:
            model.R7.add(sum(model.y[m,p,l] for l in localizaciones) <= Disponibilidad_MP.loc[p,m])
    
    #Restricción de disponibilidad de retornos
    model.R8 = pyo.ConstraintList()
    for c in clientes:
        model.R8.add(sum(model.r[l,c] for l in localizaciones) <= sum(model.x[l,c] for l in localizaciones))
    
    #Capacidad máxima de recepción de materia prima
    model.R9 = pyo.ConstraintList()
    for l in localizaciones:
        model.R9.add(sum(model.y[m,p,l] for m in m_primas for p in proveedores) <= model.w[l] * Capacidad_MP.loc[l,"Capacidad MP (ton)"])

    #Capacidad máxima de recepción de retornos
    model.R10 = pyo.ConstraintList()
    for l in localizaciones:
        model.R10.add(sum(model.r[l,c] for c in clientes) <= model.v[l] * Capacidad_r.loc[l,"Capacidad retornos (ton)"])
    
    #Selección de la localización de la planta
    model.R11 =pyo.ConstraintList()
    for l in localizaciones:
        model.R11.add(model.w[l] + model.v[l] <= 2)
    
    #Restricción de capacidad de producción
    model.R12 = pyo.ConstraintList()
    for l in localizaciones:
        model.R12.add(sum(model.x[l,c] for c in clientes) <= Cap_prod)
        
    
   # max_ec = 236041927119.4
    #min_ec = 21691956180.9

    #min_env = 68753.6
    #max_env = 370279.1

    #max_soc = 193549
    #min_soc = 0

    #model.R13 = pyo.Constraint(expr= beneficio == max_ec*0.6)
    #model.R14 = pyo.Constraint(expr= prioridad == max_soc*0.4)

#     model.R13 = pyo.Constraint(expr= beneficio == 71627283830.1599)
#     model.R14 = pyo.Constraint(expr= prioridad == 136774.626666667)



        
        
    
    """===============================================Ejecución del solver=============================================="""
    opt = pyo.SolverFactory("glpk")
    results = opt.solve(model, tee = False)
    
    print("Condicion del resultado: ",results.solver.termination_condition)
    
    """===============================================Impresión de resultados==========================================="""
    
    """---------------------------Resultado infactible---------------------------"""
    if results.solver.termination_condition == TerminationCondition.infeasible:
        fo_value = 0
        fo_status = False
        if imprimir == True:
            if fo == 1:
                print("\n Infeasible \n")
                print("\n El beneficio máximo obtenido será de\033[1m $%.1f \033[1m"%(fo_value))
                print("\n La instancia no presenta una solución factible \n")
                
            if fo == 2:
                print("\n Infeasible \n")
                print("\n El impacto ambiental mínimo causado será de\033[1m %.1f (tons)\033[1m"%(fo_value))
                print("\n La instancia no presenta una solución factible \n")

            if fo == 3:
                print("\n Infeasible \n")
                print("\n La colaboración máxima obtenida será de\033[1m %.1f (tons)\033[1m"%(fo_value))
                print("\n La instancia no presenta una solución factible \n")
                
            if fo == 4:
                print("\n Infeasible \n")
                print("\n La función multi-objetivo obtenida será de\033[1m %.1f\033[1m"%(fo_value))
                print("\n La instancia no presenta una solución factible \n")
                
    """---------------------------Resultado no acotado---------------------------"""
    if results.solver.termination_condition == TerminationCondition.unbounded:
        fo_value = "Infinito"
        fo_status = False
        if imprimir == True:
            if fo == 1:
                print("\n Unbounded")
                print("\n El beneficio máximo obtenido será de\033[1m $%s\033[1m"%(fo_value))
                print("\n La instancia no presenta una solución acotada \n")
                
            if fo == 2:
                print("\n Unbounded")
                print("\n El impacto ambiental mínimo causado será de\033[1m -%s (tons)\033[1m"%(fo_value))
                print("\n La instancia no presenta una solución acotada \n")

            if fo == 3:
                print("\n Unbounded")
                print("\n La colaboración máxima obtenida será de\033[1m %s (tons)\033[1m"%(fo_value))
                print("\n La instancia no presenta una solución acotada \n")
                
            if fo == 4:
                print("\n Unbounded")
                print("\n La función multi-objetivo obtenida será de\033[1m -%s\033[1m"%(fo_value))
                print("\n La instancia no presenta una solución acotada \n")
    
    """---------------------------Resultado factible---------------------------"""
    if results.solver.termination_condition == TerminationCondition.optimal:
        fo_status = True
        fo_value = pyo.value(model.FO)
        
        prod_PLA_total = sum(pyo.value(model.x[l, c]) for l in localizaciones for c in clientes)
        prod_COMPOST_total = sum(pyo.value(model.z[p, l]) for p in proveedores for l in localizaciones)
        print("\nProducción total de PLA: {:.2f} tons".format(prod_PLA_total))
        print("Producción total de COMPOST: {:.2f} tons".format(prod_COMPOST_total))

        
        
        """---------------------------Imprimir layout económico---------------------------"""
        if imprimir == True and fo == 1:
            i=0
            j=0
            for l in localizaciones:
                if pyo.value(model.w[l])>0:
                    print("La combinación óptima de instalación de plantas de polimerización de PLA es en la \033[1m ubicación %s \033[0m"%(l))
                    i=i+1

            for l in localizaciones:
                if pyo.value(model.v[l])>0:
                    print("La combinación óptima de instalación de plantas de Compostaje es en la \033[1m ubicación %s \033[0m"%(l))
                    j+=1

            if i == 0:
                print("\033[1m Ninguna localización \033[0m representa una inversión rentable para Polimerización de PLA")

            if j == 0:
                print("\033[1m Ninguna localización \033[0m representa una inversión rentable para Compostaje de PLA")

            for l in localizaciones:
                for c in clientes:
                    if pyo.value(model.x[l,c]) > 0:
                        print("La cantidad óptima del producto a transportar desde la \033[1mlocalización %s\033[1m es de\033[1m %.1f tons \033[0m hacia \033[1m %s \033[0m"%(l,pyo.value(model.x[l,c]), c))

            for l in localizaciones:
                for c in clientes:
                    if pyo.value(model.r[l,c]) > 0:
                        print("La cantidad óptima de retornos a transportar desde el \033[1mcliente %s\033[1m es de\033[1m %.1f tons \033[0m hacia la localización \033[1m %s \033[0m"%(c,pyo.value(model.r[l,c]), l))

            for m in m_primas:
                for p in proveedores:
                    for l in localizaciones:
                        if pyo.value(model.y[m,p,l])>0:
                            print("La cantidad óptima de la materia prima \033[1m %s \033[0m  a transportar desde el\033[1m %s\033[0m es de\033[1m %.1f tons \033[0m hacia \033[1m %s \033[0m"%(m,p,pyo.value(model.y[m,p,l]),l))

            for p in proveedores:
                    for l in localizaciones:
                        if pyo.value(model.z[p,l])>0:
                            print("La cantidad óptima de Compost a transportar desde la localización\033[1m %s\033[0m es de\033[1m %.1f tons \033[0m hacia el proveedor \033[1m %s \033[0m"%(l,pyo.value(model.z[p,l]),p))

            print("\n El beneficio máximo obtenido será de\033[1m $%.1f\033[1m"%(fo_value))

            emisiones_totales = pyo.value(emisiones_transp_totales + E_prod_totales)
            colaboracion_local = pyo.value(prioridad)

            print("\n Las emisiones totales producidas fueron de\033[1m %.1f tons \033[1m"%(emisiones_totales))
            print("\n La colaboración local lograda fue de\033[1m %.1f tons \033[1m"%(colaboracion_local))
            
        """---------------------------Imprimir layout ambiental---------------------------"""
        if imprimir == True and fo ==2:
            i=0
            j=0
            for l in localizaciones:
                if pyo.value(model.w[l])>0:
                    print("La combinación óptima de instalación de plantas de polimerización de PLA es en la \033[1m ubicación %s \033[0m"%(l))
                    i=i+1

            for l in localizaciones:
                if pyo.value(model.v[l])>0:
                    print("La combinación óptima de instalación de plantas de Compostaje es en la \033[1m ubicación %s \033[0m"%(l))
                    j+=1

            if i == 0:
                print("\033[1m Ninguna localización \033[0m representa una inversión rentable para Polimerización de PLA")

            if j == 0:
                print("\033[1m Ninguna localización \033[0m representa una inversión rentable para Compostaje de PLA")

            for l in localizaciones:
                for c in clientes:
                    if pyo.value(model.x[l,c]) > 0:
                        print("La cantidad óptima del producto a transportar desde la \033[1mlocalización %s\033[1m es de\033[1m %.1f tons \033[0m hacia \033[1m %s \033[0m"%(l,pyo.value(model.x[l,c]), c))

            for l in localizaciones:
                for c in clientes:
                    if pyo.value(model.r[l,c]) > 0:
                        print("La cantidad óptima de retornos a transportar desde el \033[1mcliente %s\033[1m es de\033[1m %.1f tons \033[0m hacia la localización \033[1m %s \033[0m"%(c,pyo.value(model.r[l,c]), l))

            for m in m_primas:
                for p in proveedores:
                    for l in localizaciones:
                        if pyo.value(model.y[m,p,l])>0:
                            print("La cantidad óptima de la materia prima \033[1m %s \033[0m  a transportar desde el\033[1m %s\033[0m es de\033[1m %.1f tons \033[0m hacia \033[1m %s \033[0m"%(m,p,pyo.value(model.y[m,p,l]),l))

            for p in proveedores:
                    for l in localizaciones:
                        if pyo.value(model.z[p,l])>0:
                            print("La cantidad óptima de Compost a transportar desde la localización\033[1m %s\033[0m es de\033[1m %.1f tons \033[0m hacia el proveedor \033[1m %s \033[0m"%(l,pyo.value(model.z[p,l]),p))

            print("\n El impacto mínimo obtenido será de \033[1m %.1f (tons) \033[1m"%(fo_value))
            
            beneficio = pyo.value(beneficio)
            colaboracion_local = pyo.value(prioridad)

            print("\n El beneficio económico obtenido fue de\033[1m $%.1f \033[1m"%(beneficio))
            print("\n La colaboración local lograda fue de\033[1m %.1f tons \033[1m"%(colaboracion_local))
        """---------------------------Imprimir layout social---------------------------"""
        if imprimir == True and fo ==3:
            i=0
            j=0
            for l in localizaciones:
                if pyo.value(model.w[l]) > 0:
                    print("La combinación óptima de instalación de plantas de polimerización de PLA es en la \033[1m ubicación %s \033[0m"%(l))
                    i=i+1
                    
            for l in localizaciones:
                if pyo.value(model.v[l]) > 0:
                    print("La combinación óptima de instalación de plantas de Compostaje es en la \033[1m ubicación %s \033[0m"%(l))
                    j+=1

            if i == 0:
                print("\033[1m Ninguna localización \033[0m representa una inversión rentable para Polimerización de PLA")

            if j == 0:
                print("\033[1m Ninguna localización \033[0m representa una inversión rentable para Compostaje de PLA")

            for l in localizaciones:
                for c in clientes:
                    if pyo.value(model.x[l,c]) > 0:
                        print("La cantidad óptima del producto a transportar desde la \033[1mlocalización %s\033[1m es de\033[1m %.1f tons \033[0m hacia \033[1m %s \033[0m"%(l,pyo.value(model.x[l,c]), c))

            for l in localizaciones:
                for c in clientes:
                    if pyo.value(model.r[l,c]) > 0:
                        print("La cantidad óptima de retornos a transportar desde el \033[1mcliente %s\033[1m es de\033[1m %.1f tons \033[0m hacia la localización \033[1m %s \033[0m"%(c,pyo.value(model.r[l,c]), l))

            for m in m_primas:
                for p in proveedores:
                    for l in localizaciones:
                        if pyo.value(model.y[m,p,l]) > 0:
                            print("La cantidad óptima de la materia prima \033[1m %s \033[0m  a transportar desde el\033[1m %s\033[0m es de\033[1m %.1f tons \033[0m hacia \033[1m %s \033[0m"%(m,p,pyo.value(model.y[m,p,l]),l))

            for p in proveedores:
                    for l in localizaciones:
                        if pyo.value(model.z[p,l]) > 0:
                            print("La cantidad óptima de Compost a transportar desde la localización\033[1m %s\033[0m es de\033[1m %.1f tons \033[0m hacia el proveedor \033[1m %s \033[0m"%(l,pyo.value(model.z[p,l]),p))

            print("\n La cooperación máxima con proveedores locales es de \033[1m %.1f (tons) \033[1m"%(fo_value))
            beneficio = pyo.value(beneficio)
            emisiones_totales = pyo.value(emisiones_transp_totales + E_prod_totales)

            print("\n El beneficio económico obtenido fue de\033[1m $%.1f \033[1m"%(beneficio))
            print("\n Las emisiones totales producidas fueron de\033[1m %.1f tons \033[1m"%(emisiones_totales))
            
        """---------------------------Imprimir layout multiobjetivo---------------------------"""
        if imprimir == True and fo == 4:
            i=0
            j=0
            for l in localizaciones:
                if pyo.value(model.w[l])>0:
                    print("La combinación óptima de instalación de plantas de polimerización de PLA es en la \033[1m ubicación %s \033[0m"%(l))
                    i=i+1

            for l in localizaciones:
                if pyo.value(model.v[l])>0:
                    print("La combinación óptima de instalación de plantas de Compostaje es en la \033[1m ubicación %s \033[0m"%(l))
                    j+=1

            if i == 0:
                print("\033[1m Ninguna localización \033[0m representa una inversión rentable para Polimerización de PLA")

            if j == 0:
                print("\033[1m Ninguna localización \033[0m representa una inversión rentable para Compostaje de PLA")

            for l in localizaciones:
                for c in clientes:
                    if pyo.value(model.x[l,c]) > 0:
                        print("La cantidad óptima del producto a transportar desde la \033[1mlocalización %s\033[1m es de\033[1m %.1f tons \033[0m hacia \033[1m %s \033[0m"%(l,pyo.value(model.x[l,c]), c))

            for l in localizaciones:
                for c in clientes:
                    if pyo.value(model.r[l,c]) > 0:
                        print("La cantidad óptima de retornos a transportar desde el \033[1m %s\033[1m es de\033[1m %.1f tons \033[0m hacia la localización \033[1m %s \033[0m"%(c,pyo.value(model.r[l,c]), l))

            for m in m_primas:
                for p in proveedores:
                    for l in localizaciones:
                        if pyo.value(model.y[m,p,l])>0:
                            print("La cantidad óptima de la materia prima \033[1m %s \033[0m  a transportar desde el\033[1m %s\033[0m es de\033[1m %.1f tons \033[0m hacia \033[1m %s \033[0m"%(m,p,pyo.value(model.y[m,p,l]),l))

            for p in proveedores:
                    for l in localizaciones:
                        if pyo.value(model.z[p,l])>0:
                            print("La cantidad óptima de Compost a transportar desde la localización\033[1m %s\033[0m es de\033[1m %.1f tons \033[0m hacia el proveedor \033[1m %s \033[0m"%(l,pyo.value(model.z[p,l]),p))

            print("\n La optimización multiobjetivo es \033[1m %.1f  \033[1m"%(fo_value))
            
            print("\n El beneficio económico es de\033[1m $ %.1f\033[1m"%(pyo.value(beneficio_MO)))
            print("\n La diferencia del beneficio económico es de\033[1m $ %.1f\033[1m"%(pyo.value(beneficio_MO)-FO_list[0]))
            print("\n El beneficio económico cambió en un\033[1m %.1f por ciento\033[1m"%(((pyo.value(beneficio_MO)-FO_list[0])/FO_list[0])*100))
            
            print("\n El impacto ambiental es de\033[1m %.1f (tons)\033[1m"%(pyo.value(E_total_MO)))
            print("\n La diferencia del impacto ambiental es de\033[1m %.1f (tons)\033[1m"%(pyo.value(E_total_MO)-FO_list[1]))
            print("\n El impacto ambiental cambió en un\033[1m %.1f por ciento\033[1m"%(((pyo.value(E_total_MO)-FO_list[1])/FO_list[1])*100))
            
            print("\n La colaboración local es de\033[1m %.1f (tons)\033[1m"%(pyo.value(prioridad_MO)))
            print("\n La diferencia de la colaboración local es de\033[1m %.1f (tons)\033[1m"%(pyo.value(prioridad_MO)-FO_list[2]))
            print("\n La colaboración local cambió en un\033[1m %.1f por ciento\033[1m"%(((pyo.value(prioridad_MO)-FO_list[2])/FO_list[2])*100))
        
    return fo_value,fo_status

#Configuración de ejecución
funciones = [1,2,3,4]
res_list = []

w = [1/3, 1/3, 1/3]
show = True
hide = False

for fo in funciones:
    print("\n======================================Resultados de Modelo %i======================================\n"%(fo))
    #print("Res_list: ",res_list)
    resultados = model_1(fo, show, w, res_list) 
    fo1 = resultados[0]
    fo_status = resultados[1]
    res_list.append(resultados[0])


======================================Resultados de Modelo 1======================================

Condicion del resultado:  optimal

Producción total de PLA: 162238.00 tons
Producción total de COMPOST: 146014.20 tons
La combinación óptima de instalación de plantas de polimerización de PLA es en la  ubicación A 
La combinación óptima de instalación de plantas de polimerización de PLA es en la  ubicación C 
La combinación óptima de instalación de plantas de polimerización de PLA es en la  ubicación D 
La combinación óptima de instalación de plantas de polimerización de PLA es en la  ubicación E 
La combinación óptima de instalación de plantas de polimerización de PLA es en la  ubicación F 
La combinación óptima de instalación de plantas de polimerización de PLA es en la  ubicación G 
La combinación óptima de instalación de plantas de polimerización de PLA es en la  ubicación H 
La combinación óptima de instalación de plantas de polimerización de PLA es en la  ubicación I 
La combinaci

Condicion del resultado:  optimal

Producción total de PLA: 32447.60 tons
Producción total de COMPOST: 18783.06 tons
La combinación óptima de instalación de plantas de polimerización de PLA es en la  ubicación B 
La combinación óptima de instalación de plantas de polimerización de PLA es en la  ubicación E 
La combinación óptima de instalación de plantas de polimerización de PLA es en la  ubicación F 
La combinación óptima de instalación de plantas de polimerización de PLA es en la  ubicación H 
La combinación óptima de instalación de plantas de Compostaje es en la  ubicación B 
La combinación óptima de instalación de plantas de Compostaje es en la  ubicación F 
La combinación óptima de instalación de plantas de Compostaje es en la  ubicación H 
La cantidad óptima del producto a transportar desde la localización B es de 6528.0 tons  hacia  Cliente2 
La cantidad óptima del producto a transportar desde la localización E es de 4469.6 tons  hacia  Cliente15 
La cantidad óptima del producto

Condicion del resultado:  optimal

Producción total de PLA: 32447.60 tons
Producción total de COMPOST: 29202.84 tons
La combinación óptima de instalación de plantas de polimerización de PLA es en la  ubicación C 
La combinación óptima de instalación de plantas de polimerización de PLA es en la  ubicación J 
La combinación óptima de instalación de plantas de Compostaje es en la  ubicación B 
La combinación óptima de instalación de plantas de Compostaje es en la  ubicación D 
La combinación óptima de instalación de plantas de Compostaje es en la  ubicación J 
La cantidad óptima del producto a transportar desde la localización C es de 6528.0 tons  hacia  Cliente2 
La cantidad óptima del producto a transportar desde la localización C es de 12513.0 tons  hacia  Cliente9 
La cantidad óptima del producto a transportar desde la localización J es de 1365.6 tons  hacia  Cliente1 
La cantidad óptima del producto a transportar desde la localización J es de 12041.0 tons  hacia  Cliente11 
La cantid

### Resultados de modelos

In [16]:
#Configuración de ejecución
funciones = [1,2,3,4]#,4]
res_list = []

w = [1/3, 1/3, 1/3]
show = True
hide = False

for fo in funciones:
    print("\n======================================Resultados de Modelo %i======================================\n"%(fo))
    #print("Res_list: ",res_list)
    resultados = model_1(fo, show, w, res_list) 
    fo1 = resultados[0]
    fo_status = resultados[1]
    res_list.append(resultados[0])


======================================Resultados de Modelo 1======================================

Condicion del resultado:  optimal

Producción total de PLA: 162238.00 tons
Producción total de COMPOST: 146014.20 tons
La combinación óptima de instalación de plantas de polimerización de PLA es en la  ubicación A 
La combinación óptima de instalación de plantas de polimerización de PLA es en la  ubicación C 
La combinación óptima de instalación de plantas de polimerización de PLA es en la  ubicación D 
La combinación óptima de instalación de plantas de polimerización de PLA es en la  ubicación E 
La combinación óptima de instalación de plantas de polimerización de PLA es en la  ubicación F 
La combinación óptima de instalación de plantas de polimerización de PLA es en la  ubicación G 
La combinación óptima de instalación de plantas de polimerización de PLA es en la  ubicación H 
La combinación óptima de instalación de plantas de polimerización de PLA es en la  ubicación I 
La combinaci

Condicion del resultado:  optimal

Producción total de PLA: 32447.60 tons
Producción total de COMPOST: 18783.06 tons
La combinación óptima de instalación de plantas de polimerización de PLA es en la  ubicación B 
La combinación óptima de instalación de plantas de polimerización de PLA es en la  ubicación E 
La combinación óptima de instalación de plantas de polimerización de PLA es en la  ubicación F 
La combinación óptima de instalación de plantas de polimerización de PLA es en la  ubicación H 
La combinación óptima de instalación de plantas de Compostaje es en la  ubicación B 
La combinación óptima de instalación de plantas de Compostaje es en la  ubicación F 
La combinación óptima de instalación de plantas de Compostaje es en la  ubicación H 
La cantidad óptima del producto a transportar desde la localización B es de 6528.0 tons  hacia  Cliente2 
La cantidad óptima del producto a transportar desde la localización E es de 4469.6 tons  hacia  Cliente15 
La cantidad óptima del producto

Condicion del resultado:  optimal

Producción total de PLA: 32447.60 tons
Producción total de COMPOST: 29202.84 tons
La combinación óptima de instalación de plantas de polimerización de PLA es en la  ubicación C 
La combinación óptima de instalación de plantas de polimerización de PLA es en la  ubicación J 
La combinación óptima de instalación de plantas de Compostaje es en la  ubicación B 
La combinación óptima de instalación de plantas de Compostaje es en la  ubicación D 
La combinación óptima de instalación de plantas de Compostaje es en la  ubicación J 
La cantidad óptima del producto a transportar desde la localización C es de 6528.0 tons  hacia  Cliente2 
La cantidad óptima del producto a transportar desde la localización C es de 12513.0 tons  hacia  Cliente9 
La cantidad óptima del producto a transportar desde la localización J es de 1365.6 tons  hacia  Cliente1 
La cantidad óptima del producto a transportar desde la localización J es de 12041.0 tons  hacia  Cliente11 
La cantid